In [61]:
import cv2
import mediapipe as mp
import numpy as np
import sys
sys.path.insert(0,'../../functions')
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
import os
from heel_crop1 import heel_crop

# from tensorflow import keras
# from tensorflow.keras import layers
# from tensorflow.keras.models import Sequential

In [62]:

# Specify the location of the file with the video to be read
# Capture frames from a saved video
filename_r = '../../Data/videos/exercise_stock_video3.mp4'
# filename_r = 4
filename_r = '../../../../Material/hill_raise/IMG_3719.MOV'
# Specify the location where the new video with detections will be written
# filename_w = '../Data/videos/exercise_stock_video3_wRepCount.mp4'

cap = cv2.VideoCapture(filename_r)

# Get video frame dimensions and fps
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
frame_size = (frame_width,frame_height)
fps = cap.get(5)

# # Create a video writer object
# output = cv2.VideoWriter(filename_w, cv2.VideoWriter_fourcc(*'XVID'), fps, frame_size)

frame_count = 0
frame_count_trim = 0

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.9, min_tracking_confidence=0.5, enable_segmentation=True) as pose:

    fps = cap.get(5)
    frame_width  = int(cap.get(3))
    frame_height = int(cap.get(4))

    while cap.isOpened():
        ret, frame = cap.read()

        # Rotate the video feed if its coming from the phone webcam
        if filename_r == 4:
            frame=cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)


        # Stop the code when the video is over
        if not ret:
            print('Failed to grab a frame')
            break

        # Count frames and calculate time in video
        frame_count += 1
        time = frame_count / fps #[s]

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        image_trimmed = frame

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            right_ear = [landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            left_heel = [landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y]
            right_heel = [landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].y]
            left_footIndex = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]
            right_footIndex = [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y]
            
        
            points = [left_ankle,left_heel,left_footIndex]
            [image_trimmed, frame_count_trim] = heel_crop(points, frame, frame_count_trim)
            
        except:
            pass
        

        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )


        # Display and write the video
        if ret == True:
            cv2.imshow('Mediapipe Feed', image)
            # output.write(image)

            try:
                cv2.imshow('Right foot', image_trimmed)
                # if frame_count_trim % 5 == 0:
                    # cv2.imwrite(f'/home/rafik/PROJECTS/pose1/Material/hill_raise/cropped_feet/left_foot/left_foot_{frame_count_trim}.jpg', image_trimmed)
            except:
                print('No foot in sight')

            # Pause or stop the video when instructed
            key = cv2.waitKey(5)
            # Stop by pressing 'q'
            if key == ord('q'):
                break
            # Pause by pressing 'w', resume by pressing any other key
            if key == ord('w'):
                cv2.waitKey(-1)


    cap.release()
    cv2.destroyAllWindows()


I0000 00:00:1701556954.369788  158802 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1701556954.371567  192045 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3), renderer: AMD Radeon Graphics (renoir, LLVM 15.0.7, DRM 3.54, 6.5.0-13-generic)
qt.qpa.xcb: QXcbConnection: XCB error: 146 (Unknown), sequence: 38249, resource id: 0, major code: 139 (Unknown), minor code: 20
qt.qpa.xcb: QXcbConnection: XCB error: 146 (Unknown), sequence: 38254, resource id: 1329, major code: 139 (Unknown), minor code: 20


In [56]:
print(os.getcwd())

/home/rafik/PROJECTS/pose1/MediaPipePoseEstimation-eschar/SquatMaster/ML_training/test1
